# <span style="color:pink;font-family:Cascadia Code;font-size:20pt">Analysis of glutathionylation in their structural context using StructureMap </span> 
##### <span style="color:blue;font-family:Cascadia Code;font-size:14pt"> Adapted from https://academic.oup.com/nar/article/50/D1/D471/6426061#325777216 </span>


### Prepare the environment

In [2]:
#Requires py3.8-3.10
import structuremap.utils

In [3]:
structuremap.utils.set_logger()

In [4]:
from structuremap.processing import download_alphafold_cif, download_alphafold_pae, format_alphafold_data, annotate_accessibility, get_smooth_score
from structuremap.processing import perform_enrichment_analysis, perform_enrichment_analysis_per_protein
from structuremap.processing import annotate_proteins_with_idr_pattern, get_extended_flexible_pattern
from structuremap.processing import get_avg_3d_dist, get_avg_1d_dist, get_proximity_pvals, evaluate_ptm_colocalization
from structuremap.plotting import plot_enrichment, plot_ptm_colocalization

In [5]:
import pandas as pd
import numpy as np
import os
import re
import plotly.express as px
import tqdm
import tempfile

In [6]:
from accessory_functions import *

#### Download AlphaFold data

In [7]:
output_dir = ("c:/Users/marga/OneDrive/Desktop/summer project/")
cif_dir = os.path.join(output_dir, 'tutorial_cif')
pae_dir = os.path.join(output_dir, 'tutorial_pae')

In [8]:
protein_data = pd.read_csv(r"C:\Users\marga\Downloads\output_all_noU2OS_fixed.csv") # change for different pdresult outputs 
protein_ids = sorted(protein_data["protein_id"].unique())
print(protein_ids)

['A0A024QZ33', 'A0A024R4E5', 'A0A024R571', 'A0A024RCR6', 'A0A087WT44', 'A0A087WT92', 'A0A087WTF6', 'A0A087WU57', 'A0A087WVQ6', 'A0A087WVZ6', 'A0A087WVZ9', 'A0A087WWW9', 'A0A087WX29', 'A0A087WXM8', 'A0A087WY31', 'A0A087WYF8', 'A0A087WYR0', 'A0A087WYT3', 'A0A087WZ13', 'A0A087X0M4', 'A0A087X0P9', 'A0A087X0W8', 'A0A087X0X3', 'A0A087X142', 'A0A087X1C5', 'A0A087X1R1', 'A0A087X1Z3', 'A0A087X251', 'A0A087X253', 'A0A087X2D5', 'A0A087X2I1', 'A0A096LNZ9', 'A0A096LP25', 'A0A0A0MQT0', 'A0A0A0MQV3', 'A0A0A0MQV6', 'A0A0A0MQW0', 'A0A0A0MQX8', 'A0A0A0MR02', 'A0A0A0MRA6', 'A0A0A0MRJ3', 'A0A0A0MRM8', 'A0A0A0MRM9', 'A0A0A0MRY0', 'A0A0A0MSA9', 'A0A0A0MSB8', 'A0A0A0MT01', 'A0A0A0MT68', 'A0A0A0MTL6', 'A0A0A6YYL4', 'A0A0B4J1R7', 'A0A0B4J2D5', 'A0A0C4DFM1', 'A0A0C4DFX9', 'A0A0C4DG63', 'A0A0C4DG89', 'A0A0C4DGA2', 'A0A0C4DGC9', 'A0A0D9SFB1', 'A0A0D9SFE4', 'A0A0D9SGE8', 'A0A0G2JH37', 'A0A0G2JIW1', 'A0A0G2JLB3', 'A0A0G2JMX7', 'A0A0G2JN84', 'A0A0G2JRX8', 'A0A0J9YWM9', 'A0A0J9YYL3', 'A0A0U1RQR5', 'A0A0U1RQV4', 'A0A0

In [23]:
valid_proteins_cif, invalid_proteins_cif, existing_proteins_cif = download_alphafold_cif(
    proteins=protein_ids,
    out_folder=cif_dir)

  2%|▏         | 65/2651 [00:01<00:38, 66.51it/s]

100%|██████████| 2651/2651 [00:24<00:00, 108.29it/s]

2023-08-18 18:50:34> Valid proteins: 0
2023-08-18 18:50:34> Invalid proteins: 158
2023-08-18 18:50:34> Existing proteins: 2493


In [24]:
valid_proteins_pae, invalid_proteins_pae, existing_proteins_pae = download_alphafold_pae(
    proteins=protein_ids,
    out_folder=pae_dir, 
    )

100%|██████████| 2651/2651 [00:28<00:00, 92.28it/s] 

2023-08-18 18:51:03> Valid proteins: 0
2023-08-18 18:51:03> Invalid proteins: 158
2023-08-18 18:51:03> Existing proteins: 2493


In [25]:
# Test if equal protein files available in cif and pae folder
test_identical_ids('C:/Users/marga/OneDrive/Desktop/summer project/tutorial_cif',
                   'C:/Users/marga/OneDrive/Desktop/summer project/tutorial_pae')

Number of unique proteins with cif and pae file:  20797


# Format AlphaFold data

In [26]:
all_proteins = valid_proteins_cif + existing_proteins_cif
all_proteins = list(set(all_proteins))

In [27]:
protein_ids= list(set(protein_ids))
len(protein_ids)

# Extract the first part before the comma for each item
protein_names = [item.split(',')[0] for item in protein_ids]

print(protein_names)

len(protein_names)

['O95817', 'Q13813', 'Q96ST2', 'O75179', 'A0A804HKC5', 'H9KV75', 'O14979', 'Q9Y3P9', 'B1AK63', 'O43617', 'P37837', 'Q8NHH9', 'Q96F86', 'D6RCB9', 'P61224', 'Q8IYJ3', 'B3KS98', 'E7EN96', 'A0A0B4J2D5', 'Q6ZTR5', 'Q70CQ2', 'P51508', 'P04350', 'Q8N4Q0', 'O75683', 'Q14157', 'P12883', 'Q96RS6', 'O94776', 'Q9Y613', 'A0A0C4DGA2', 'P07384', 'A8MYV0', 'Q92547', 'O15144', 'P35658', 'Q6ZUT6', 'Q9Y2R4', 'P46063', 'O95639', 'F5H5N1', 'P07737', 'O14613', 'P07741', 'Q9UK76', 'A0A0A0MSB8', 'P62750', 'Q9BRT6', 'A0A804HKG9', 'P32321', 'Q06546', 'Q14527', 'P06756', 'Q16740', 'Q14651', 'Q9H3Q1', 'Q9BVG4', 'Q5C9Z4', 'Q9NRX4', 'Q13255', 'P09972', 'O15226', 'Q99470', 'A0A669KB05', 'P61289', 'O75340', 'P43487', 'P23246', 'M0QXT0', 'E7EVG6', 'Q9H488', 'P02751', 'P31937', 'Q15024', 'A0A8I5KSD1', 'K7ENR6', 'Q8N635', 'P46939', 'Q00839', 'Q01082', 'Q9H2U2', 'O15091', 'Q8NBJ7', 'P49757', 'H0YEH2', 'M0R0P8', 'Q13823', 'Q9NYV4', 'Q9BXJ9', 'Q13564', 'P60842', 'P51532', 'Q9H0D6', 'Q9H6Z4', 'P19367', 'P40925', 'Q9H9B4', '

2651

In [28]:
%%time
alphafold_annotation = format_alphafold_data(directory=cif_dir, 
                                             protein_ids=protein_names)

  0%|          | 0/20797 [00:00<?, ?it/s]

100%|██████████| 20797/20797 [14:06<00:00, 24.57it/s] 


CPU times: total: 4min 51s
Wall time: 14min 16s


In [29]:
alphafold_annotation.to_csv('c:/Users/marga/OneDrive/Desktop/summer project/alphafold_data/alphafold_annotation_own.csv', index=False)

In [30]:
# alphafold_annotation = pd.read_csv('data/alphafold_data/alphafold_annotation.csv')

In [31]:
alphafold_annotation.columns

Index(['protein_id', 'protein_number', 'AA', 'position', 'quality',
       'x_coord_c', 'x_coord_ca', 'x_coord_cb', 'x_coord_n', 'y_coord_c',
       'y_coord_ca', 'y_coord_cb', 'y_coord_n', 'z_coord_c', 'z_coord_ca',
       'z_coord_cb', 'z_coord_n', 'secondary_structure', 'structure_group',
       'BEND', 'HELX', 'STRN', 'TURN', 'unstructured'],
      dtype='object')

# Annotate amino acid exposure metric

### Full sphere exposure

In [32]:
%%time
full_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=24, 
    max_angle=180, 
    error_dir=pae_dir)

  0%|          | 0/2493 [00:00<?, ?it/s]

100%|██████████| 2493/2493 [03:24<00:00, 12.20it/s]


CPU times: total: 1min 4s
Wall time: 3min 25s


In [33]:
full_sphere_exposure.to_csv('c:/Users/marga/OneDrive/Desktop/summer project/alphafold_data/full_sphere_exposure_own.csv', index=False)

In [34]:
# full_sphere_exposure = pd.read_csv('data/alphafold_data/full_sphere_exposure.csv')

In [35]:
alphafold_accessibility = alphafold_annotation.merge(
    full_sphere_exposure, how='left', on=['protein_id','AA','position'])

### Half sphere exposure

In [36]:
%%time
half_sphere_exposure = annotate_accessibility(
    df=alphafold_annotation, 
    max_dist=12, 
    max_angle=70, 
    error_dir=pae_dir)

100%|██████████| 2493/2493 [07:15<00:00,  5.72it/s]  


CPU times: total: 18.7 s
Wall time: 7min 18s


In [37]:
half_sphere_exposure.to_csv('c:/Users/marga/OneDrive/Desktop/summer project/alphafold_data/half_sphere_exposure_own.csv', index=False)

In [38]:
# half_sphere_exposure = pd.read_csv('data/alphafold_data/half_sphere_exposure.csv')

In [39]:
alphafold_accessibility = alphafold_accessibility.merge(
    half_sphere_exposure, how='left', on=['protein_id','AA','position'])

In [40]:
alphafold_accessibility['high_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae <= 5, 1, 0)
alphafold_accessibility['low_acc_5'] = np.where(alphafold_accessibility.nAA_12_70_pae > 5, 1, 0)

# Smooth scores for unstructured regions

In [41]:
alphafold_accessibility_smooth = get_smooth_score(alphafold_accessibility, 
                                                  np.array(['quality', 'nAA_24_180_pae']), 
                                                  [10])

100%|██████████| 2493/2493 [00:10<00:00, 240.75it/s]


# Annotate IDRs >> IDR_benchmark notebook

In [42]:
alphafold_accessibility_smooth['IDR'] = np.where(alphafold_accessibility_smooth['nAA_24_180_pae_smooth10']<=34.27, 1, 0)

### Visualize pPAE cutoff

In [43]:
bincount = np.unique(alphafold_accessibility_smooth[alphafold_accessibility_smooth.IDR==0].nAA_12_70_pae.values, return_counts=True) # 
bincount_df = pd.DataFrame({'pPSE':bincount[0],'count':bincount[1]})
bincount_df['cutoff'] = np.where(bincount_df.pPSE<=5, 'high exposure', 'low exposure')

In [44]:
bincount_df.to_csv('c:/Users/marga/OneDrive/Desktop/summer project/alphafold_data/pPSE_bincount_df_own.csv', index=False)

In [45]:
pPSE_cut = px.bar(bincount_df, x='pPSE',y='count', color='cutoff', 
                  color_discrete_map={'high exposure':'rgb(177, 63, 100)',
                                      'low exposure':'grey'}, 
                  template="simple_white",
                  width=500, height=300)
pPSE_cut = pPSE_cut.update_layout(legend=dict(
    title='',
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99
))
config={'toImageButtonOptions': {'format': 'svg', 'filename':'pPAE_cutoff'}}

pPSE_cut.show(config=config)

# Find short unstructured regions within large folded domains

In [46]:
alphafold_accessibility_smooth_pattern = annotate_proteins_with_idr_pattern(alphafold_accessibility_smooth,
                                                min_structured_length = 80, 
                                                max_unstructured_length = 20)

100%|██████████| 2493/2493 [00:08<00:00, 292.52it/s]


In [47]:
alphafold_accessibility_smooth_pattern_ext = get_extended_flexible_pattern(
    alphafold_accessibility_smooth_pattern, 
    ['flexible_pattern'], [5])

100%|██████████| 2493/2493 [00:06<00:00, 391.74it/s]


In [48]:
alphafold_accessibility_smooth_pattern_ext[0:3]

,protein_id,protein_number,AA,position,quality,x_coord_c,x_coord_ca,x_coord_cb,x_coord_n,y_coord_c,...,unstructured,nAA_24_180_pae,nAA_12_70_pae,high_acc_5,low_acc_5,quality_smooth10,nAA_24_180_pae_smooth10,IDR,flexible_pattern,flexible_pattern_extended_5
0,A0A024QZ33,1,M,1,62.81,29.208,28.215,27.418,28.792,-9.402,...,1,8,0,1,0,84.091818,18.000000,1,0,0
1,A0A024QZ33,1,K,2,78.75,32.264,31.377,32.209,30.346,-9.984,...,1,12,0,1,0,84.412500,18.416667,1,0,0
2,A0A024QZ33,1,Q,3,85.62,32.948,33.644,33.977,32.818,-7.963,...,0,16,0,1,0,84.568462,18.692308,1,0,0


In [49]:
proteins_with_pattern = alphafold_accessibility_smooth_pattern_ext[alphafold_accessibility_smooth_pattern_ext.flexible_pattern==1].protein_id.unique()

textfile = open("c:/Users/marga/OneDrive/Desktop/summer project/short_idrs/proteins_with_pattern.txt", "w")
for element in proteins_with_pattern:
    textfile.write(element + "\n") 

all_proteins = alphafold_accessibility_smooth_pattern_ext.protein_id.unique()

textfile = open("c:/Users/marga/OneDrive/Desktop/summer project/short_idrs/all_proteins.txt", "w")
for element in all_proteins:
    textfile.write(element + "\n")
textfile.close()

In [50]:
print(len(proteins_with_pattern))

335


In [51]:
print(len(all_proteins))

2493


# Kinase loop annotation

Here we read kinase substructures annotated in [KinaseMD](https://doi.org/10.1093/nar/gkaa945). 

https://bioinfo.uth.edu/kmd/

In [52]:
loop_info = pd.read_csv('c:/Users/marga/OneDrive/Desktop/summer project/kinase_substructures/kinasemd_substrucutres.tsv', sep='\t')


In [53]:
len(loop_info.uniprot_id.unique())

388

In [54]:
loop_list = extract_loop_annotation(loop_info)

In [55]:
alphafold_loops = alphafold_accessibility_smooth_pattern_ext.copy(deep=True)

for l in loop_list:
    alphafold_loops = alphafold_loops.merge(l, how='left', on=['protein_id','position'])
    
alphafold_loops = alphafold_loops.fillna(0)


In [56]:
alphafold_loops[0:3]

,protein_id,protein_number,AA,position,quality,x_coord_c,x_coord_ca,x_coord_cb,x_coord_n,y_coord_c,...,high_acc_5,low_acc_5,quality_smooth10,nAA_24_180_pae_smooth10,IDR,flexible_pattern,flexible_pattern_extended_5,gloop,aloop,achelix
0,A0A024QZ33,1,M,1,62.81,29.208,28.215,27.418,28.792,-9.402,...,1,0,84.091818,18.000000,1,0,0,0.0,0.0,0.0
1,A0A024QZ33,1,K,2,78.75,32.264,31.377,32.209,30.346,-9.984,...,1,0,84.412500,18.416667,1,0,0,0.0,0.0,0.0
2,A0A024QZ33,1,Q,3,85.62,32.948,33.644,33.977,32.818,-7.963,...,1,0,84.568462,18.692308,1,0,0,0.0,0.0,0.0


# Annotate PTM data

In [57]:
def annotate_ptm_data(alphafold_data: pd.DataFrame) -> pd.DataFrame:

    alphafold_data_annotated = alphafold_data.copy(deep=True)
    ptm_file = pd.read_csv(r"C:\Users\marga\Downloads\output_all_noU2OS_fixed.csv")
    alphafold_data_annotated = alphafold_data_annotated.merge(ptm_file, how='left', on=['protein_id','position'])
    alphafold_data_annotated = alphafold_data_annotated.fillna(0)
    return(alphafold_data_annotated)

alphafold_ptms = annotate_ptm_data(alphafold_loops)

column_to_remove = 'AA_y'
alphafold_ptms.drop(column_to_remove, axis=1, inplace=True)  # axis=1 indicates columns, inplace=True modifies the DataFrame in place
#print(alphafold_ptms)
current_column_name = 'AA_x'
new_column_name = 'AA'
alphafold_ptms.rename(columns={current_column_name: new_column_name}, inplace=True)
print(alphafold_ptms)
alphafold_ptms.to_csv(r"c:\Users\marga\OneDrive\Desktop\summer project\alphafold_ptms.csv")

         protein_id  protein_number AA  position  quality  x_coord_c  \
0        A0A024QZ33               1  M         1    62.81     29.208   
1        A0A024QZ33               1  K         2    78.75     32.264   
2        A0A024QZ33               1  Q         3    85.62     32.948   
3        A0A024QZ33               1  T         4    84.56     30.678   
4        A0A024QZ33               1  K         5    84.69     31.980   
...             ...             ... ..       ...      ...        ...   
1439642      X6RM59            2493  L       327    98.44     -9.367   
1439643      X6RM59            2493  Q       328    97.31    -11.690   
1439644      X6RM59            2493  K       329    96.50    -11.700   
1439645      X6RM59            2493  I       330    97.31     -8.548   
1439646      X6RM59            2493  L       331    95.81     -7.460   

         x_coord_ca  x_coord_cb  x_coord_n  y_coord_c  ...  \
0            28.215      27.418     28.792     -9.402  ...   
1          

pPSE calculations

In [59]:
# Plot for ox PTM
ox_data = alphafold_ptms[alphafold_ptms["Oxidation"]==1]
ox_bincount = np.unique(ox_data['nAA_12_70_pae'].values, return_counts=True)
ox_bincount_df = pd.DataFrame({'pPSE': ox_bincount[0], 'count': ox_bincount[1]})
ox_bincount_df['cutoff'] = np.where(ox_bincount_df.pPSE <= 5, 'high exposure', 'low exposure')
pPSE_cut_ox = px.bar(ox_bincount_df, x='pPSE', y='count', color='cutoff',
                     color_discrete_map={'high exposure': 'rgb(177, 63, 100)', 'low exposure': 'grey'},
                     template="simple_white", width=500, height=300)
pPSE_cut_ox = pPSE_cut_ox.update_layout(legend=dict(title='', yanchor="top", y=0.99, xanchor="right", x=0.99))
config_ox = {'toImageButtonOptions': {'format': 'svg', 'filename': 'pPAE_cutoff_ox'}}
print("Oxidation")
pPSE_cut_ox.show(config=config_ox)

Oxidation


In [60]:
# Plot for phos PTM
phos_data = alphafold_ptms[(alphafold_ptms['Phosphorylation'] == 1)]
phos_bincount = np.unique(phos_data['nAA_12_70_pae'].values, return_counts=True)
phos_bincount_df = pd.DataFrame({'pPSE': phos_bincount[0], 'count': phos_bincount[1]})
phos_bincount_df['cutoff'] = np.where(phos_bincount_df.pPSE <= 5, 'high exposure', 'low exposure')
pPSE_cut_phos = px.bar(phos_bincount_df, x='pPSE', y='count', color='cutoff',
                     color_discrete_map={'high exposure': 'rgb(177, 63, 100)', 'low exposure': 'grey'},
                     template="simple_white", width=500, height=300)
pPSE_cut_phos = pPSE_cut_phos.update_layout(legend=dict(title='', yanchor="top", y=0.99, xanchor="right", x=0.99))
config_phos = {'toImageButtonOptions': {'format': 'svg', 'filename': 'pPAE_cutoff_phos'}}
print("Phosphorylation")
pPSE_cut_phos.show(config=config_phos)


Phosphorylation


In [61]:
# N-term carbamyl
carb_data = alphafold_ptms[(alphafold_ptms['Carbamyl'] == 1)]
carb_bincount = np.unique(carb_data['nAA_12_70_pae'].values, return_counts=True)
carb_bincount_df = pd.DataFrame({'pPSE': carb_bincount[0], 'count': carb_bincount[1]})
carb_bincount_df['cutoff'] = np.where(carb_bincount_df.pPSE <= 5, 'high exposure', 'low exposure')
pPSE_cut_carb = px.bar(carb_bincount_df, x='pPSE', y='count', color='cutoff',
                     color_discrete_map={'high exposure': 'rgb(177, 63, 100)', 'low exposure': 'grey'},
                     template="simple_white", width=500, height=300)
pPSE_cut_carb = pPSE_cut_carb.update_layout(legend=dict(title='', yanchor="top", y=0.99, xanchor="right", x=0.99))
config_carb = {'toImageButtonOptions': {'format': 'svg', 'filename': 'pPAE_cutoff_carb'}}
print("Carbamylation")
pPSE_cut_carb.show(config=config_carb)

Carbamylation


In [62]:
# nitrosyl
nitro_data = alphafold_ptms[(alphafold_ptms['Nitrosyl'] == 1)]
nitro_bincount = np.unique(nitro_data['nAA_12_70_pae'].values, return_counts=True)
nitro_bincount_df = pd.DataFrame({'pPSE': nitro_bincount[0], 'count': nitro_bincount[1]})
nitro_bincount_df['cutoff'] = np.where(nitro_bincount_df.pPSE <= 5, 'high exposure', 'low exposure')
pPSE_cut_nitro = px.bar(nitro_bincount_df, x='pPSE', y='count', color='cutoff',
                     color_discrete_map={'high exposure': 'rgb(177, 63, 100)', 'low exposure': 'grey'},
                     template="simple_white", width=500, height=300)
pPSE_cut_nitro = pPSE_cut_nitro.update_layout(legend=dict(title='', yanchor="top", y=0.99, xanchor="right", x=0.99))
config_nitro = {'toImageButtonOptions': {'format': 'svg', 'filename': 'pPAE_cutoff_nitro'}}
print("Nitrosylation")
pPSE_cut_nitro.show(config=config_nitro)

Nitrosylation


In [63]:
# Cysteinylation
cys_data = alphafold_ptms[alphafold_ptms["Cysteinyl"]==1]
cys_bincount = np.unique(cys_data['nAA_12_70_pae'].values, return_counts=True)
cys_bincount_df = pd.DataFrame({'pPSE': cys_bincount[0], 'count': cys_bincount[1]})
cys_bincount_df['cutoff'] = np.where(cys_bincount_df.pPSE <= 5, 'high exposure', 'low exposure')
pPSE_cut_cys = px.bar(cys_bincount_df, x='pPSE', y='count', color='cutoff',
                     color_discrete_map={'high exposure': 'rgb(177, 63, 100)', 'low exposure': 'grey'},
                     template="simple_white", width=500, height=300)
pPSE_cut_cys = pPSE_cut_cys.update_layout(legend=dict(title='', yanchor="top", y=0.99, xanchor="right", x=0.99))
config_cys = {'toImageButtonOptions': {'format': 'svg', 'filename': 'pPAE_cutoff_cys'}}
print("Cysteinylation")
pPSE_cut_cys.show(config=config_cys)

Cysteinylation


# Assess kinase annotations

Kinases with structural annotation that have an alphafold structure

In [64]:
len(alphafold_ptms[(alphafold_ptms.gloop==1) | (alphafold_ptms.aloop==1) | (alphafold_ptms.achelix==1)].protein_id.unique())

31

Gloops

In [65]:
len(alphafold_ptms[(alphafold_ptms.gloop==1)].protein_id.unique())

22

In [66]:
len(alphafold_ptms[(alphafold_ptms.gloop==1) & (alphafold_ptms.flexible_pattern==1)].protein_id.unique())

0

In [67]:
len(alphafold_ptms[(alphafold_ptms.gloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1)].protein_id.unique())

0

Calpha-Helix

In [68]:
len(alphafold_ptms[(alphafold_ptms.achelix==1)].protein_id.unique())

28

In [69]:
len(alphafold_ptms[(alphafold_ptms.achelix==1) & (alphafold_ptms.flexible_pattern==1)].protein_id.unique())

0

In [70]:
len(alphafold_ptms[(alphafold_ptms.achelix==1) & (alphafold_ptms.flexible_pattern_extended_5==1)].protein_id.unique())

0

Aloops

In [71]:
len(alphafold_ptms[(alphafold_ptms.aloop==1)].protein_id.unique())

27

In [72]:
len(alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern==1)].protein_id.unique())

4

In [73]:
len(alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1)].protein_id.unique())

5

In [74]:
print('Proteins with overlap between Aloop and extended short IDR with phosphosites:')
len(alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Phosphorylation == 1)].protein_id.unique())
print('Number of phosphosites in the overlap:')
alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Phosphorylation == 1)].shape[0]
# Count the total number of occurrences with Phosphorylation
total_phosphorylation_occurrences = alphafold_ptms[alphafold_ptms.Phosphorylation == 1].shape[0]

# Count the number of occurrences with overlap between Aloop and extended short IDR with Phosphorylation
overlap_phosphorylation_occurrences = alphafold_ptms[(alphafold_ptms.aloop == 1) & 
                                                     (alphafold_ptms.flexible_pattern_extended_5 == 1) & 
                                                     (alphafold_ptms.Phosphorylation == 1)].shape[0]

# ratio = overlap_phosphorylation_occurrences / total_phosphorylation_occurrences
# print(f'Ratio of Phosphorylation occurrences in the overlap: {ratio:.2f}')


Proteins with overlap between Aloop and extended short IDR with phosphosites:
Number of phosphosites in the overlap:


In [75]:
print('Proteins with overlap between Aloop and extended short IDR with glut:')
len(alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Glutathionylation == 1)].protein_id.unique())

print('Number of glut in the overlap:')
alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Glutathionylation == 1)].shape[0]
# Count the total number of occurrences with Glutathionylation
total_glutathionylation_occurrences = alphafold_ptms[alphafold_ptms.Glutathionylation == 1].shape[0]

# Count the number of occurrences with overlap between Aloop and extended short IDR with Glutathionylation
overlap_glutathionylation_occurrences = alphafold_ptms[(alphafold_ptms.aloop == 1) & 
                                                       (alphafold_ptms.flexible_pattern_extended_5 == 1) & 
                                                       (alphafold_ptms.Glutathionylation == 1)].shape[0]

# Calculate the ratio
ratio = overlap_glutathionylation_occurrences / total_glutathionylation_occurrences

print(f'Ratio of Glutathionylation occurrences in the overlap: {ratio:.2f}')


Proteins with overlap between Aloop and extended short IDR with glut:
Number of glut in the overlap:


Ratio of Glutathionylation occurrences in the overlap: 0.00


In [76]:
print('Proteins with overlap between Aloop and extended short IDR with carb:')
len(alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Carbamyl == 1)].protein_id.unique())

print('Number of carb in the overlap:')
alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Carbamyl == 1)].shape[0]
# Count the total number of occurrences with Carbamyl
total_carbamyl_occurrences = alphafold_ptms[alphafold_ptms.Carbamyl == 1].shape[0]

# Count the number of occurrences with overlap between Aloop and extended short IDR with Carbamyl
overlap_carbamyl_occurrences = alphafold_ptms[(alphafold_ptms.aloop == 1) & 
                                              (alphafold_ptms.flexible_pattern_extended_5 == 1) & 
                                              (alphafold_ptms.Carbamyl == 1)].shape[0]

# Calculate the ratio
ratio = overlap_carbamyl_occurrences / total_carbamyl_occurrences

print(f'Ratio of Carbamyl occurrences in the overlap: {ratio:.2f}')

Proteins with overlap between Aloop and extended short IDR with carb:
Number of carb in the overlap:
Ratio of Carbamyl occurrences in the overlap: 0.00


In [77]:
print('Proteins with overlap between Aloop and extended short IDR with ox:')
len(alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Oxidation == 1)].protein_id.unique())

print('Number of ox in the overlap:')
alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Oxidation == 1)].shape[0]
# Count the total number of occurrences with Oxidation
total_oxidation_occurrences = alphafold_ptms[alphafold_ptms.Oxidation == 1].shape[0]

# Count the number of occurrences with overlap between Aloop and extended short IDR with Oxidation
overlap_oxidation_occurrences = alphafold_ptms[(alphafold_ptms.aloop == 1) & 
                                                (alphafold_ptms.flexible_pattern_extended_5 == 1) & 
                                                (alphafold_ptms.Oxidation == 1)].shape[0]

# Calculate the ratio
ratio = overlap_oxidation_occurrences / total_oxidation_occurrences

print(f'Ratio of Oxidation occurrences in the overlap: {ratio:.2f}')

Proteins with overlap between Aloop and extended short IDR with ox:
Number of ox in the overlap:
Ratio of Oxidation occurrences in the overlap: 0.00


In [78]:
print('Proteins with overlap between Aloop and extended short IDR with nitrosyl:')
len(alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Nitrosyl == 1)].protein_id.unique())

print('Number of nitro in the overlap:')
alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Nitrosyl == 1)].shape[0]
# Count the total number of occurrences with Oxidation
total_nitro_occurrences = alphafold_ptms[alphafold_ptms.Nitrosyl == 1].shape[0]

# Count the number of occurrences with overlap between Aloop and extended short IDR with Oxidation
overlap_nitro_occurrences = alphafold_ptms[(alphafold_ptms.aloop == 1) & 
                                                (alphafold_ptms.flexible_pattern_extended_5 == 1) & 
                                                (alphafold_ptms.Nitrosyl == 1)].shape[0]

# Calculate the ratio
ratio = overlap_nitro_occurrences / total_nitro_occurrences

print(f'Ratio of Nitrosyl occurrences in the overlap: {ratio:.2f}')


Proteins with overlap between Aloop and extended short IDR with nitrosyl:
Number of nitro in the overlap:
Ratio of Nitrosyl occurrences in the overlap: 0.00


In [79]:
print('Proteins with overlap between Aloop and extended short IDR with cys:')
len(alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Cysteinyl == 1)].protein_id.unique())

print('Number of cys in the overlap:')
alphafold_ptms[(alphafold_ptms.aloop==1) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Cysteinyl == 1)].shape[0]
# Count the total number of occurrences with Oxidation
total_cys_occurrences = alphafold_ptms[alphafold_ptms.Cysteinyl == 1].shape[0]

# Count the number of occurrences with overlap between Aloop and extended short IDR with Oxidation
overlap_cys_occurrences = alphafold_ptms[(alphafold_ptms.aloop == 1) & 
                                                (alphafold_ptms.flexible_pattern_extended_5 == 1) & 
                                                (alphafold_ptms.Cysteinyl == 1)].shape[0]

# Calculate the ratio
ratio = overlap_cys_occurrences / total_cys_occurrences

print(f'Ratio of Cysteinyl occurrences in the overlap: {ratio:.2f}')


Proteins with overlap between Aloop and extended short IDR with cys:
Number of cys in the overlap:
Ratio of Cysteinyl occurrences in the overlap: 0.00


### Get list of all PTMs in short IDRs

In [80]:
potentially_interesting_sites = alphafold_ptms[(alphafold_ptms.flexible_pattern_extended_5==1) & 
                                               ((alphafold_ptms.Phosphorylation==1) | (alphafold_ptms.Nitrosyl==1) | 
                                                (alphafold_ptms.Glutathionylation==1) | 
                                                (alphafold_ptms.Carbamyl==1)|(alphafold_ptms.Cysteinyl))][[
    'protein_id','position','IDR', 'flexible_pattern','flexible_pattern_extended_5','aloop',
    "Phosphorylation", "Glutathionylation", 'Carbamyl', "Nitrosyl", "Cysteinyl"]].reset_index(drop=True)

In [81]:
potentially_interesting_sites[0:3]

,protein_id,position,IDR,flexible_pattern,flexible_pattern_extended_5,aloop,Phosphorylation,Glutathionylation,Carbamyl,Nitrosyl,Cysteinyl
0,A0A087X1C5,338,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0
1,O75369,2502,1,1,1,0.0,0.0,1.0,0.0,0.0,0.0
2,O75369,2502,1,1,1,0.0,0.0,1.0,0.0,0.0,0.0


In [82]:
potentially_interesting_sites.to_csv('c:/Users/marga/OneDrive/Desktop/summer project/mods_in_short_idrs/potentially_interesting_sites_own.tsv',
                                     sep='\t', index=False)

### Get list of all short IDRs

In [83]:
short_idr_info = extract_short_IDR_info(alphafold_ptms)
short_idr_info[0:4]

,protein_id,IDR_start,IDR_end,sequence
0,A0A087WU57,174,187,LTQAAIMEKLVAYS
1,A0A087X1C5,332,345,RVSPGCPIVGTHVC
2,A0A0A0MRM8,623,639,SSTNNNKDTKQKAGKLS
3,A0A0C4DFM1,129,143,LYSNRDSDDKKKEKD


In [84]:
short_idr_info.to_csv('c:/Users/marga/OneDrive/Desktop/summer project/mods_in_short_idrs/short_idr_info_own.tsv',
                      sep='\t', index=False)


# Get hotlist of PTMs in short IDRs

In [86]:
res = list()
for ptm_type in ["Phosphorylation", "Oxidation", "Glutathionylation",'Carbamyl', "Nitrosyl", "Cysteinyl"]:
    n_in_shortIDR = alphafold_ptms[(alphafold_ptms.flexible_pattern==1) & (alphafold_ptms[ptm_type]==1)].shape[0]
    n_in_shortIDRextention = alphafold_ptms[(alphafold_ptms.flexible_pattern==0) & (alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms[ptm_type]==1)].shape[0]
    res.append([ptm_type, n_in_shortIDR, n_in_shortIDRextention])
res = pd.DataFrame(res, columns=['ptm','Short IDR','5 AA extension'])
res = pd.melt(res, id_vars=['ptm'], value_vars=['Short IDR','5 AA extension'])

col_map = dict({'Short IDR':"#66985E",'5 AA extension':"#b2cbae"})

In [87]:
res.to_csv("c:/Users/marga/OneDrive/Desktop/summer project/mods_in_short_idrs/short_idr_site_summary_own.tsv", sep='\t', index=False)

In [88]:
fig = px.bar(res,x='ptm',y='value', color='variable', text='value',
             color_discrete_map=col_map,
            barmode='group')
fig = fig.update_traces(textfont_size=8, textangle=0, 
                        textposition="outside", #['inside', 'outside', 'auto', 'none']
                        cliponaxis=False)
fig = fig.update_yaxes(col=1, title='count')
fig = fig.update_yaxes(matches=None,showticklabels=True, col=2)
fig = fig.update_xaxes(matches=None, title='')
fig = fig.update_layout(template="simple_white", 
                        width=800, height=350, 
                       )

config={'toImageButtonOptions': {'format': 'svg', 'filename':'ptms_in_shortIDRs'}}

fig.show(config=config)
# considering glut has been selected for in these analyses more relevant perhaps to look at the % in IDR? 

In [89]:
alphafold_ptms[(alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Phosphorylation==1)].shape[0]

1

In [90]:
#alphafold_ptms[(alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.functional_score>0)].shape[0]

In [91]:
#alphafold_ptms[(alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.functional_score>0.5)].shape[0]

In [92]:
alphafold_ptms[(alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Glutathionylation==1)].shape[0]

18

In [93]:
alphafold_ptms[(alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Oxidation==1)].shape[0]

93

In [94]:
alphafold_ptms[(alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Carbamyl==1)].shape[0]

0

In [95]:
alphafold_ptms[(alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Nitrosyl==1)].shape[0]

1

In [96]:
alphafold_ptms[(alphafold_ptms.flexible_pattern_extended_5==1) & (alphafold_ptms.Cysteinyl==1)].shape[0]

1

# Generate PTM site dict

In [16]:
def generate_ptm_site_dict(alphafold_df):
    all_ptm_datasets = ['Phosphorylation', 'Glutathionylation', 'Carbamyl', 
    'Oxidation', "Nitrosyl", "Cysteinyl"]
    ptm_dict = {}
    for d in all_ptm_datasets:
        df_d = alphafold_df[alphafold_df[d] == 1]
        unique_aa = list(np.unique(df_d.AA.values))
        ptm_dict.update({d: unique_aa})

    return(ptm_dict)

In [98]:
print(alphafold_ptms)

         protein_id  protein_number AA  position  quality  x_coord_c  \
0        A0A024QZ33               1  M         1    62.81     29.208   
1        A0A024QZ33               1  K         2    78.75     32.264   
2        A0A024QZ33               1  Q         3    85.62     32.948   
3        A0A024QZ33               1  T         4    84.56     30.678   
4        A0A024QZ33               1  K         5    84.69     31.980   
...             ...             ... ..       ...      ...        ...   
1439642      X6RM59            2493  L       327    98.44     -9.367   
1439643      X6RM59            2493  Q       328    97.31    -11.690   
1439644      X6RM59            2493  K       329    96.50    -11.700   
1439645      X6RM59            2493  I       330    97.31     -8.548   
1439646      X6RM59            2493  L       331    95.81     -7.460   

         x_coord_ca  x_coord_cb  x_coord_n  y_coord_c  ...  \
0            28.215      27.418     28.792     -9.402  ...   
1          

In [99]:
#column_to_remove = 'AA_y'
#alphafold_ptms.drop(column_to_remove, axis=1, inplace=True)  # axis=1 indicates columns, inplace=True modifies the DataFrame in place
#current_column_name = 'AA_x'
#new_column_name = 'AA'
#alphafold_ptms.rename(columns={current_column_name: new_column_name}, inplace=True)


In [17]:
ptm_site_dict = generate_ptm_site_dict(alphafold_ptms)

ptm_site_dict


{'Phosphorylation': ['A',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'K',
  'L',
  'M',
  'N',
  'P',
  'Q',
  'R',
  'S',
  'T',
  'V'],
 'Glutathionylation': ['A',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'K',
  'L',
  'M',
  'N',
  'P',
  'Q',
  'R',
  'S',
  'T',
  'V',
  'W',
  'Y'],
 'Carbamyl': ['E', 'G', 'M'],
 'Oxidation': ['A',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'K',
  'L',
  'M',
  'N',
  'P',
  'Q',
  'R',
  'S',
  'T',
  'V',
  'W',
  'Y'],
 'Nitrosyl': ['A',
  'C',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'K',
  'L',
  'M',
  'N',
  'P',
  'Q',
  'R',
  'S',
  'T',
  'V',
  'Y'],
 'Cysteinyl': ['A',
  'D',
  'E',
  'F',
  'G',
  'H',
  'I',
  'K',
  'L',
  'M',
  'N',
  'P',
  'Q',
  'R',
  'S',
  'T',
  'V',
  'W',
  'Y']}

# Perform enrichment analysis

In [101]:
enrichment_res = perform_enrichment_analysis(
    df=alphafold_ptms, 
    ptm_types=list(ptm_site_dict.keys()), 
    rois=['IDR'], 
    quality_cutoffs=[0],
    ptm_site_dict=ptm_site_dict)
enrichment_res


,quality_cutoff,ptm,roi,n_aa_ptm,n_aa_roi,n_ptm_in_roi,n_ptm_not_in_roi,n_naked_in_roi,n_naked_not_in_roi,oddsr,p,p_adj_bf,p_adj_bh
0,0,Phosphorylation,IDR,369,507092,273,96,506819,882144,4.949690,2.788372e-48,1.673023e-47,5.576745e-48
0,0,Glutathionylation,IDR,3857,518633,832,3025,517801,917989,0.487610,1.622217e-84,9.733301e-84,4.866650e-84
0,0,Carbamyl,IDR,901,98608,491,410,98117,145422,1.774939,1.106426e-17,6.638555e-17,1.659639e-17
0,0,Oxidation,IDR,20203,518633,4828,15375,513805,905639,0.553489,8.150693e-305,4.890416e-304,4.890416e-304
0,0,Nitrosyl,IDR,64,515383,16,48,515367,910370,0.588817,6.869444e-02,4.121666e-01,6.869444e-02
0,0,Cysteinyl,IDR,155,513884,29,126,513855,902561,0.404262,2.265636e-06,1.359382e-05,2.718764e-06


In [102]:
# Get % glu in IDRs
res_glu_t = enrichment_res[enrichment_res.ptm=="Glutathionylation"]
res_glu_t.n_ptm_not_in_roi.values/res_glu_t.n_aa_ptm.values

array([0.78428831])

In [103]:
plot_enrichment(data=enrichment_res,
                ptm_select=["Phosphorylation", 'Glutathionylation','Oxidation','Carbamyl', "Nitrosyl", "Cysteinyl"],
                roi_select=['IDR'])

In [104]:
enrichment_res.to_csv("C:/Users/marga/OneDrive/Desktop/summer project/ptm_enrichment/ptm_enrichment_idrs_own.tsv", sep="\t", index=False)

In [105]:
enrichment_res = perform_enrichment_analysis(
    df=alphafold_ptms[alphafold_ptms.IDR==0], 
    ptm_types=list(ptm_site_dict.keys()), 
    rois=['high_acc_5'], 
    quality_cutoffs=[0],
    ptm_site_dict=ptm_site_dict)

In [106]:
plot_enrichment(data=enrichment_res,
                ptm_select=['Phosphorylation','Glutathionylation','Oxidation','Carbamylation', "Nitrosylation", "Cysteinylation"],
                roi_select=['high_acc_5'],
                )

In [107]:
enrichment_res.to_csv("c:/Users/marga/OneDrive/Desktop/summer project/ptm_enrichment/ptm_enrichment_high_acc_5_own.tsv", sep="\t", index=False)

# Kinase motifs

In [109]:
kinase_motifs = pd.read_csv('c:/Users/marga/OneDrive/Desktop/summer project/kinase_substructures/kinase_motifs.txt', 
                            sep='\t', 
                            names=['enzyme','start','sty','end'])
kinase_motifs = kinase_motifs.astype('str')
kinase_motifs[0:3] 

,enzyme,start,sty,end
0,Akt kinase substrate motif,"R,X,R,X,X",ST,FL
1,Akt kinase substrate motif,"R,X,R,X,X",ST,nan
2,Akt kinase substrate motif,"G,R,A,R,T,ST",S,FAE


In [110]:
kinase_motifs['start'] = kinase_motifs['start'].apply(string_to_motif)
kinase_motifs['sty'] = kinase_motifs['sty'].apply(string_to_motif)
kinase_motifs['end'] = kinase_motifs['end'].apply(string_to_motif)
kinase_motifs['motif'] = kinase_motifs['start']+kinase_motifs['sty']+kinase_motifs['end']
kinase_motifs['mod_pos'] = [s.count(']') for s in kinase_motifs['start']]
kinase_motifs[0:3] 

,enzyme,start,sty,end,motif,mod_pos
0,Akt kinase substrate motif,[R][A-Z][R][A-Z][A-Z],[ST],[FL],[R][A-Z][R][A-Z][A-Z][ST][FL],5
1,Akt kinase substrate motif,[R][A-Z][R][A-Z][A-Z],[ST],,[R][A-Z][R][A-Z][A-Z][ST],5
2,Akt kinase substrate motif,[G][R][A][R][T][ST],[S],[FAE],[G][R][A][R][T][ST][S][FAE],6


In [111]:
from structuremap.processing import extract_motifs_in_proteome

In [112]:
kinase_motif_res = extract_motifs_in_proteome(alphafold_df=alphafold_ptms, motif_df=kinase_motifs)

100%|██████████| 2493/2493 [01:09<00:00, 35.79it/s] 


In [113]:
# Format observed kinase motifs for merging with alphafold_ptms
kinase_motif_res['kinase_motif'] = 1
kinase_motif_res_sub = kinase_motif_res[['protein_id','position','AA','kinase_motif']].drop_duplicates()
alphafold_motifs = alphafold_ptms.merge(kinase_motif_res_sub, how='left', on=['protein_id','position','AA'])
alphafold_motifs = alphafold_motifs.fillna(0)
# alphafold_motifs[0:3]

In [114]:
# test if any are enriched in kinase motifs
enrichment_all_inMotif = perform_enrichment_analysis(
    df=alphafold_motifs, 
    ptm_types=['Phosphorylation','Glutathionylation','Oxidation',
    'Carbamyl', "Nitrosyl", "Cysteinyl"], 
    rois=['kinase_motif'], 
    ptm_site_dict=ptm_site_dict,
    quality_cutoffs=[0])
# inspect
enrichment_all_inMotif

,quality_cutoff,ptm,roi,n_aa_ptm,n_aa_roi,n_ptm_in_roi,n_ptm_not_in_roi,n_naked_in_roi,n_naked_not_in_roi,oddsr,p,p_adj_bf,p_adj_bh
0,0,Phosphorylation,kinase_motif,369,80154,16,353,80138,1308825,0.740267,2.649646e-01,1.000000e+00,3.974470e-01
0,0,Glutathionylation,kinase_motif,3857,93781,220,3637,93561,1342229,0.867783,4.271917e-02,2.563150e-01,8.543834e-02
0,0,Oxidation,kinase_motif,20203,93781,689,19514,93092,1326352,0.503060,4.151741e-86,2.491044e-85,2.491044e-85
0,0,Carbamyl,kinase_motif,901,0,0,901,0,243539,NaN,1.000000e+00,1.000000e+00,1.000000e+00
0,0,Nitrosyl,kinase_motif,64,93781,10,54,93771,1331966,2.630455,8.697737e-03,5.218642e-02,2.609321e-02
0,0,Cysteinyl,kinase_motif,155,93781,11,144,93770,1322646,1.077482,7.465905e-01,1.000000e+00,8.959086e-01


In [115]:
plot_enrichment(data=enrichment_all_inMotif,
                ptm_select=['Phosphorylation','Glutathionylation','Oxidation',
                'Carbamyl', "Nitrosyl", "Cysteinyl"],
                roi_select=['kinase_motif']
                )

In [116]:
enrichment_all_inMotif.to_csv("c:/Users/marga/OneDrive/Desktop/summer project/ptm_enrichment/enrichment_all_inMotif_own.tsv", sep="\t", index=False)

In [117]:
# test if sites in motifs are enriched in IDRs
enrichment_all_inMotif_inIDR = perform_enrichment_analysis(
    df=alphafold_motifs[alphafold_motifs.kinase_motif==1], 
    ptm_types=["Phosphorylation", 'Glutathionylation','Oxidation',
    'Carbamyl', "Nitrosyl", "Cysteinyl"], 
    rois=['IDR'], 
    ptm_site_dict=ptm_site_dict,
    quality_cutoffs=[0])

# inspect
enrichment_all_inMotif_inIDR

,quality_cutoff,ptm,roi,n_aa_ptm,n_aa_roi,n_ptm_in_roi,n_ptm_not_in_roi,n_naked_in_roi,n_naked_not_in_roi,oddsr,p,p_adj_bf,p_adj_bh
0,0,Phosphorylation,IDR,16,39440,10,6,39430,40708,1.720686,3.254256e-01,1.000000e+00,4.378164e-01
0,0,Glutathionylation,IDR,220,42793,57,163,42736,50825,0.415883,2.312270e-09,1.387362e-08,1.387362e-08
0,0,Oxidation,IDR,689,42793,252,437,42541,50551,0.685238,1.513194e-06,9.079163e-06,4.539581e-06
0,0,Carbamyl,IDR,0,0,0,0,0,0,NaN,1.000000e+00,1.000000e+00,1.000000e+00
0,0,Nitrosyl,IDR,10,42793,3,7,42790,50981,0.510610,3.623268e-01,1.000000e+00,4.378164e-01
0,0,Cysteinyl,IDR,11,42793,3,8,42790,50980,0.446775,3.648470e-01,1.000000e+00,4.378164e-01


In [118]:
plot_enrichment(data=enrichment_all_inMotif_inIDR,
                ptm_select=['Phosphorylation','Glutathionylation','Oxidation',
                'Carbamyl', "Nitrosyl", "Cysteinyl"],
                roi_select=['IDR']
                )

In [119]:
enrichment_all_inMotif_inIDR.to_csv("C:/Users/marga/OneDrive/Desktop/summer project/ptm_enrichment/enrichment_all_inMotif_inIDR_own.tsv", sep="\t", index=False)

In [120]:
# test if phosphosites in motifs are enriched in highAcc
enrichment_all_inMotif_inHighAcc = perform_enrichment_analysis(
    df=alphafold_motifs[alphafold_motifs.kinase_motif==1], 
    ptm_types=['Phosphorylation','Glutathionylation','Oxidation',
    'Carbamyl', "Nitrosyl", "Cysteinyl"], 
    rois=['high_acc_5'], 
    ptm_site_dict=ptm_site_dict,
    quality_cutoffs=[0])

# inspect
enrichment_all_inMotif_inHighAcc

,quality_cutoff,ptm,roi,n_aa_ptm,n_aa_roi,n_ptm_in_roi,n_ptm_not_in_roi,n_naked_in_roi,n_naked_not_in_roi,oddsr,p,p_adj_bf,p_adj_bh
0,0,Phosphorylation,high_acc_5,16,70591,15,1,70576,9562,2.032277,0.711529,1.000000,0.853835
0,0,Glutathionylation,high_acc_5,220,79072,168,52,78904,14657,0.600139,0.002068,0.012408,0.012408
0,0,Oxidation,high_acc_5,689,79072,592,97,78480,14612,1.136320,0.269360,1.000000,0.538719
0,0,Carbamyl,high_acc_5,0,0,0,0,0,0,NaN,1.000000,1.000000,1.000000
0,0,Nitrosyl,high_acc_5,10,79072,8,2,79064,14707,0.744055,0.662208,1.000000,0.853835
0,0,Cysteinyl,high_acc_5,11,79072,11,0,79061,14709,inf,0.232688,1.000000,0.538719


In [121]:
plot_enrichment(data=enrichment_all_inMotif_inHighAcc,
                ptm_select=['Phosphorylation','Glutathionylation','Oxidation',
                'Carbamyl', "Nitrosyl", "Cysteinyl"],
                roi_select=['high_acc_5']
                )

In [122]:
enrichment_all_inMotif_inHighAcc.to_csv("c:/Users/marga/OneDrive/Desktop/summer project/ptm_enrichment/enrichment_all_inMotif_inHighAcc_own.tsv", sep="\t", index=False)

# Proximity analysis

In [12]:
alphafold_ptms = pd.read_csv(r"c:\Users\marga\OneDrive\Desktop\summer project\alphafold_ptms.csv")
#column_to_remove = 'AA_y'
#alphafold_ptms.drop(column_to_remove, axis=1, inplace=True)  # axis=1 indicates columns, inplace=True modifies the DataFrame in place
#print(alphafold_ptms)
#current_column_name = 'AA_x'
#new_column_name = 'AA'
#alphafold_ptms.rename(columns={current_column_name: new_column_name}, inplace=True)

alphafold_ptms_noIDRs = alphafold_ptms[(alphafold_ptms.IDR==0) | (alphafold_ptms.flexible_pattern==1)]
print(alphafold_ptms_noIDRs)

         Unnamed: 0  protein_id  protein_number AA  position  quality  \
62               62  A0A024QZ33               1  R        60    93.94   
63               63  A0A024QZ33               1  K        61    95.38   
64               64  A0A024QZ33               1  K        62    89.88   
65               65  A0A024QZ33               1  E        63    91.31   
66               66  A0A024QZ33               1  Q        64    92.94   
...             ...         ...             ... ..       ...      ...   
1439642     1439642      X6RM59            2493  L       327    98.44   
1439643     1439643      X6RM59            2493  Q       328    97.31   
1439644     1439644      X6RM59            2493  K       329    96.50   
1439645     1439645      X6RM59            2493  I       330    97.31   
1439646     1439646      X6RM59            2493  L       331    95.81   

         x_coord_c  x_coord_ca  x_coord_cb  x_coord_n  ...  \
62          -9.224      -8.332      -8.889     -6.938  ...   

In [13]:
alphafold_ptms_onlyIDRs = alphafold_ptms[(alphafold_ptms.IDR==1) & (alphafold_ptms.flexible_pattern==0)]

## Evaluate PTM co-localization 

In [18]:
self_colocalization = evaluate_ptm_colocalization(
    df=alphafold_ptms_noIDRs, 
    ptm_target='self',
    ptm_types=['Phosphorylation', 'Glutathionylation','Oxidation',
    'Carbamyl', "Nitrosyl", "Cysteinyl"], 
    ptm_dict=ptm_site_dict,
    pae_dir=pae_dir,
    min_dist = 1,
    max_dist = 35,
    dist_step = 5)

# self_colocalization.to_csv('c:/Users/marga/OneDrive/Desktop/summer project/proximity_analysis/Fraction_of_modified_acceptor_residues_self_noIDRs_own.csv', index=False)

100%|██████████| 2352/2352 [00:27<00:00, 85.11it/s] 


In [ ]:
plot_ptm_colocalization(self_colocalization, context="3D", plot_width=1500)

In [ ]:
glu_colocalization = evaluate_ptm_colocalization(
    df=alphafold_ptms_noIDRs, 
    ptm_target='Glutathionylation',
    ptm_types=["Phosphorylation", 'Oxidation',
    'Carbamyl', "Nitrosyl", "Cysteinyl"], 
    ptm_dict=ptm_site_dict,
    pae_dir=pae_dir,
    min_dist = 0,
    max_dist = 35,
    dist_step = 5)

glu_colocalization.to_csv('c:/Users/marga/OneDrive/Desktop/summer project/proximity_analysis/Fraction_of_modified_acceptor_residues_glu_colocalization_noIDRs_own.csv', index=False)

100%|██████████| 2352/2352 [00:50<00:00, 47.02it/s] 


In [ ]:
plot_ptm_colocalization(glu_colocalization, context="3D", plot_width=1500)

## 3D PTM clusters

In [ ]:
%%time
proximity_res_pandub_mean_plus = get_proximity_pvals(
    df=alphafold_ptms_noIDRs, 
    ptm_types = ['Glutathionylation','Oxidation'], 
    ptm_site_dict = ptm_site_dict, 
    error_dir=pae_dir, 
    per_site_metric= 'mean',
    error_operation='plus',
    n_random=10000, 
    random_seed=44)

cluster_prots_pandub_df = proximity_res_pandub_mean_plus[(proximity_res_pandub_mean_plus.pvalue_3d_adj_bh <= 0.05) & (proximity_res_pandub_mean_plus.n_ptms > 3)]
cluster_prots_pandub = list(cluster_prots_pandub_df.protein_id)

cluster_prots_pandub_df

100%|██████████| 2352/2352 [27:30<00:00,  1.43it/s]  


CPU times: total: 13min 38s
Wall time: 27min 30s


,protein_id,ptm,n_ptms,pvalue_1d,pvalue_3d,pvalue_1d_adj_bh,pvalue_3d_adj_bh
1,A0A024R571,Oxidation,17.0,0.0049,0.0004,0.006482,0.000549
3,A0A087WT44,Oxidation,25.0,0.2784,0.0041,0.310016,0.005052
4,A0A087WTF6,Oxidation,4.0,0.0000,0.0000,0.000000,0.000000
5,A0A087WVZ9,Oxidation,8.0,0.0007,0.0000,0.001004,0.000000
9,A0A087WYR0,Oxidation,4.0,0.0049,0.0022,0.006482,0.002774
...,...,...,...,...,...,...,...
2113,Q9Y6Y8,Oxidation,4.0,0.0000,0.0000,0.000000,0.000000
2114,V9GZ56,Oxidation,6.0,0.0000,0.0000,0.000000,0.000000
2115,X1WI28,Glutathionylation,4.0,0.0001,0.0001,0.000163,0.000149
2116,X1WI28,Oxidation,18.0,0.0009,0.0000,0.001271,0.000000


In [ ]:
%%time
proximity_res_pandub_mean_plus = get_proximity_pvals(
    df=alphafold_ptms_noIDRs, 
    ptm_types = ['Glutathionylation','Nitrosyl'], 
    ptm_site_dict = ptm_site_dict, 
    error_dir=pae_dir, 
    per_site_metric= 'mean',
    error_operation='plus',
    n_random=10000, 
    random_seed=44)

cluster_prots_pandub_df = proximity_res_pandub_mean_plus[(proximity_res_pandub_mean_plus.pvalue_3d_adj_bh <= 0.05) & (proximity_res_pandub_mean_plus.n_ptms > 3)]
cluster_prots_pandub = list(cluster_prots_pandub_df.protein_id)

cluster_prots_pandub_df

100%|██████████| 2352/2352 [06:26<00:00,  6.08it/s]

CPU times: total: 3min 11s
Wall time: 6min 29s


,protein_id,ptm,n_ptms,pvalue_1d,pvalue_3d,pvalue_1d_adj_bh,pvalue_3d_adj_bh
14,A0A0G2JIW1,Glutathionylation,12.0,0.9981,0.0067,0.998100,0.008560
26,A0A3B3IUA2,Glutathionylation,4.0,0.0000,0.0000,0.000000,0.000000
29,A0A6I8PU73,Glutathionylation,6.0,0.0000,0.0000,0.000000,0.000000
31,A0A6Q8PGR9,Glutathionylation,10.0,0.0134,0.0169,0.017745,0.020615
33,A0A7I2V2U2,Glutathionylation,4.0,0.0050,0.0010,0.006898,0.001445
...,...,...,...,...,...,...,...
714,Q9Y3I0,Glutathionylation,4.0,0.0000,0.0000,0.000000,0.000000
716,Q9Y3U8,Glutathionylation,7.0,0.0000,0.0000,0.000000,0.000000
720,Q9Y5M8,Glutathionylation,4.0,0.0000,0.0000,0.000000,0.000000
723,Q9Y696,Glutathionylation,6.0,0.0502,0.0002,0.061857,0.000354


In [ ]:
%%time
proximity_res_pandub_mean_plus = get_proximity_pvals(
    df=alphafold_ptms_noIDRs, 
    ptm_types = ['Glutathionylation','Cysteinyl', 'Nitrosyl'], 
    ptm_site_dict = ptm_site_dict, 
    error_dir=pae_dir, 
    per_site_metric= 'mean',
    error_operation='plus',
    n_random=10000, 
    random_seed=44)

cluster_prots_pandub_df = proximity_res_pandub_mean_plus[(proximity_res_pandub_mean_plus.pvalue_3d_adj_bh <= 0.05) & (proximity_res_pandub_mean_plus.n_ptms > 3)]
cluster_prots_pandub = list(cluster_prots_pandub_df.protein_id)

cluster_prots_pandub_df

  0%|          | 0/2352 [00:00<?, ?it/s]

100%|██████████| 2352/2352 [06:45<00:00,  5.81it/s]

CPU times: total: 3min 19s
Wall time: 6min 45s


,protein_id,ptm,n_ptms,pvalue_1d,pvalue_3d,pvalue_1d_adj_bh,pvalue_3d_adj_bh
14,A0A0G2JIW1,Glutathionylation,12.0,0.9981,0.0067,0.998100,0.008677
26,A0A3B3IUA2,Glutathionylation,4.0,0.0000,0.0000,0.000000,0.000000
29,A0A6I8PU73,Glutathionylation,6.0,0.0000,0.0000,0.000000,0.000000
31,A0A6Q8PGR9,Glutathionylation,10.0,0.0134,0.0169,0.018077,0.020873
33,A0A7I2V2U2,Glutathionylation,4.0,0.0050,0.0010,0.007024,0.001483
...,...,...,...,...,...,...,...
733,Q9Y3I0,Glutathionylation,4.0,0.0000,0.0000,0.000000,0.000000
735,Q9Y3U8,Glutathionylation,7.0,0.0000,0.0000,0.000000,0.000000
739,Q9Y5M8,Glutathionylation,4.0,0.0000,0.0000,0.000000,0.000000
742,Q9Y696,Glutathionylation,6.0,0.0501,0.0000,0.062604,0.000000


In [ ]:
cluster_prots_pandub_df['protein_id'].to_csv("C:/Users/marga/OneDrive/summer project/cluster_prots_pandub_protein_ids.csv") #take these protein ids and use in DAVID

david_pandub = pd.read_csv(r"C:\Users\marga\OneDrive\Desktop\summer project\proximity_analysis\david.ncifcrf.gov_data_download_chart_6758C295E61E1691831531614.txt", sep="\t")

In [ ]:
plot_enrichment_david(david_pandub, fdr_threshold=0.01, fold_enrichment_threshold=1, count_threshold=10)